In [1]:
%pip install brainflow
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Setting Up Brainflow

In [2]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import webbrowser

/opt/anaconda3/lib/python3.13/site-packages/brainflow/board_shim.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
PARAMS = BrainFlowInputParams()
PARAMS.serial_port = "/dev/cu.usbserial-DP04VZSH"

In [4]:
try:
    board_ID = BoardIds.CYTON_DAISY_BOARD
    board = BoardShim(board_ID, PARAMS)
    board.prepare_session()
    print("Successfully connected to Cyton+Daisy board.")
except Exception as e:
    print(f"Error connecting to Cyton+Daisy board: {e}")
    input("Want to use synthetic board instead? Press Enter to continue...")
    board_ID = BoardIds.SYNTHETIC_BOARD
    board = BoardShim(board_ID, PARAMS)
    board.prepare_session()
    print("Successfully connected to synthetic board.")

board.release_session()


[2025-11-24 09:23:59.650] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}
[2025-11-24 09:23:59.651] [board_logger] [info] opening port /dev/cu.usbserial-DP04VZSH
[2025-11-24 09:23:59.674] [board_logger] [info] set_latency_res is: 0


Successfully connected to Cyton+Daisy board.


## Testing Brainflow (optional)

In [6]:
# testing sampling rate and data acquisition
print("starting stream...")
board.prepare_session()
board.start_stream()
time.sleep(5)  # collect data for 5 seconds
data = board.get_board_data()
print("stopping stream...")
board.stop_stream()
board.release_session()

starting stream...


[2025-11-13 18:47:05.581] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}
[2025-11-13 18:47:05.581] [board_logger] [info] opening port /dev/cu.usbserial-DP04VZSH
[2025-11-13 18:47:05.620] [board_logger] [info] set_latency_res is: 0


stopping stream...


In [7]:
print(type(data))
print(data.shape)  # should be (num_channels, num_samples)

<class 'numpy.ndarray'>
(32, 613)


In [8]:
print(data[BoardShim.get_eeg_channels(2)])

[[  -2142.41470604   -4416.99527705   -3994.36849289 ...   -3685.42268102
    -3768.10178376   -3697.80554745]
 [ -25720.53232438  -52571.2135519   -53365.92982601 ...  -52362.76118311
   -51957.88168405  -52696.47272783]
 [ -41755.27146521  -84238.09221245  -84748.6507593  ...  -86765.8509929
   -86307.72963854  -86976.00209427]
 ...
 [   3800.51181322     982.18035486    1478.81376491 ...    1773.16388764
     2150.93072068    1786.79845176]
 [  18064.32224087  -30428.99434912  -32853.30925623 ... -108509.00215018
  -109710.85544954 -108711.77717588]
 [   2464.72686109    4553.76560137    5143.8069515  ...    4994.58670552
     5308.38284592    4999.25822011]]


# Getting Data

In [5]:
# getting stimuli info
stimuli_info = pd.read_csv('/Users/noorelbanna/Desktop/ProjectMUSEEG/Experiment/StimuliSet/relevant_vidData.csv')
print(stimuli_info.columns.to_list())

['clip_name', 'video_url', 'clip_duration', 'rating']


In [9]:
# subject number 
subject = int(input("Enter subject ID: "))

# checking vid index
index_q = input("Enter the index of the (0-98) video to play: ")
index_q = int(index_q)

for index, row in stimuli_info.iloc[index_q:].iterrows():
    # get video info
    url = row['video_url']
    duration = row['clip_duration']  # in seconds
    rating = row['rating'] 
    print(f"Opening video: {url} for {duration} seconds")
    webbrowser.open(url)

    time.sleep(10)  # give some time for the browser to open
    print("starting stream...")
    board.prepare_session()
    board.start_stream()

    time.sleep(duration)  # collect data for the duration of the clip

    print("stopping stream...")
    board.stop_stream()
    clip_data = board.get_board_data()
    board.release_session()

    #isolate eeg channels
    EEG_CHANNELS = BoardShim.get_eeg_channels(board_ID)
    print(f"EEG channels: {EEG_CHANNELS}")
    eeg_data = clip_data[EEG_CHANNELS]
    print(f"EEG data shape: {eeg_data.shape}")  # should be (num_eeg_channels, num_samples)

    # save to file
    if eeg_data.size > 0:
        print(f"Collected {eeg_data.shape} samples for this video.")
        np.save(f"/Users/noorelbanna/Desktop/ProjectMUSEEG/Experiment/StimuliSet/{rating}EEG/video_{index}_{subject}_EEG.npy", eeg_data)
    else:
        print("No EEG data collected for this video.")

    # ask if want to continue
    cont = input("Do you want to continue to the next video? (y/n): ")
    if cont.lower() != 'y':
        break


Opening video: https://drive.google.com/file/d/0B5S5TDsmc6mAS21zX0tFZG9JeWc/view?usp=share_link&resourcekey=0-Acg3BqCvjrVxM0XJC1tGlw for 132.12 seconds
starting stream...


[2025-11-24 09:29:37.112] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}
[2025-11-24 09:29:37.114] [board_logger] [info] opening port /dev/cu.usbserial-DP04VZSH
[2025-11-24 09:29:37.129] [board_logger] [info] set_latency_res is: 0


stopping stream...
EEG channels: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
EEG data shape: (16, 16524)
Collected (16, 16524) samples for this video.
Opening video: https://drive.google.com/file/d/0B5S5TDsmc6mAS29rNWQ4U0x3ZGs/view?usp=share_link&resourcekey=0-FDxyAGJocz9pmfzqIenWww for 151.53 seconds
starting stream...


[2025-11-24 09:34:53.159] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}
[2025-11-24 09:34:53.162] [board_logger] [info] opening port /dev/cu.usbserial-DP04VZSH
[2025-11-24 09:34:53.173] [board_logger] [info] set_latency_res is: 0


stopping stream...
EEG channels: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
EEG data shape: (16, 18951)
Collected (16, 18951) samples for this video.
Opening video: https://drive.google.com/file/d/0B5S5TDsmc6mAOFZZcGNBeVBmZzA/view?usp=share_link&resourcekey=0-1PHAEc5HiW-AprK6Q1Fntg for 115.31 seconds
starting stream...


...

stopping stream...
EEG channels: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
EEG data shape: (16, 9844)
Collected (16, 9844) samples for this video.
Opening video: https://drive.google.com/file/d/0B5S5TDsmc6mAeGt3MzZfUFdNcms/view?usp=share_link&resourcekey=0-hZVnAeYrZ8n4V6Wd53N1Xw for 157.78 seconds
starting stream...


[2025-11-24 09:48:51.524] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}
[2025-11-24 09:48:51.525] [board_logger] [info] opening port /dev/cu.usbserial-DP04VZSH
[2025-11-24 09:48:51.534] [board_logger] [info] set_latency_res is: 0


stopping stream...
EEG channels: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
EEG data shape: (16, 19731)
Collected (16, 19731) samples for this video.
Opening video: https://drive.google.com/file/d/0B5S5TDsmc6mAVjd2Y3F3NWZVNVU/view?usp=share_link&resourcekey=0-hfzb82KVsTuFEgy7hbOwQg for 147.52 seconds
starting stream...


[2025-11-24 09:52:44.398] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}
[2025-11-24 09:52:44.400] [board_logger] [info] opening port /dev/cu.usbserial-DP04VZSH
[2025-11-24 09:52:44.407] [board_logger] [info] set_latency_res is: 0


stopping stream...
EEG channels: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
EEG data shape: (16, 18448)
Collected (16, 18448) samples for this video.


In [8]:
# board.stop_stream()
board.release_session()